In [2]:
import os

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv("key")

# You can now use 'api_key' in your code
print(f"Your API Key: {api_key}")

Your API Key: AQ.Ab8RN6LeT07KmJZlBGqLZerWF-tnPNjBW8LubxmcW3nxNEGLug


In [175]:
import csv
import json
import os
from pathlib import Path

import tqdm
from google import genai

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "hypnotic-bounty-474514-q4-f1a5dff58154.json"

project_id = "hypnotic-bounty-474514-q4"  # Replace with your Google Cloud project ID
region = "asia-east1"  # Replace with your desired region

client = genai.Client(
    vertexai=True, project=project_id, location=region
)

BATCH_SIZE=20

root = Path("../data/books")

index = 1

with open('embedding.csv', 'w', encoding='utf-8', newline='') as csvfile:
    # Create a csv.writer object
    writer = csv.writer(csvfile)

    # Write the header row
    #writer.writerow(['id', 'description', 'embedding'])

    for path in sorted(root.glob("book*")):
        if path.is_file():
            with path.open("r", encoding="utf-8") as f:
                data = json.load(f)
                obj_array = [str(obj) for obj in data]
                int_array = [obj.replace("\ufffd","") for obj in obj_array]
                int2_array = [obj.replace("\\x93","\"") for obj in int_array]
                strings_array = [obj.replace("\\x94","\"") for obj in int2_array]
                len_str = len(strings_array)
                for i in tqdm.tqdm(range(0, len_str, BATCH_SIZE)):
                    embeddings: list[list[float]] = []
                    response = client.models.embed_content(
                        model='gemini-embedding-001',
                        contents=strings_array[i:i+BATCH_SIZE]
                    )
                    embeddings = embeddings + [e.values for e in response.embeddings]

                    if i + BATCH_SIZE > len_str:
                        for n in range(0, len_str - i):
                            writer.writerow([str(index+i+n), strings_array[i+n], str(embeddings[n])])
                    else:
                        for n in range(0, BATCH_SIZE):
                            writer.writerow([str(index+i+n), strings_array[i+n], str(embeddings[n])])
                index += len_str
    client.close()


100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:39<00:00,  1.06s/it]


[0.0006920915329828858, -0.03979841619729996, -0.006949087604880333, -0.03953603282570839, -0.007307480089366436, 0.0020394788589328527, -0.016977788880467415, 0.008440080098807812, 0.006589439697563648, 0.007675612810999155, -0.03264548256993294, -0.024544820189476013, 0.008123588748276234, -0.02289801463484764, 0.12905758619308472, -0.006704175379127264, -0.007119165733456612, -0.0015801076078787446, 0.019186247140169144, -0.010472421534359455, 0.017694927752017975, -0.019023314118385315, -0.0006730131572112441, -0.010444100014865398, -0.008646275848150253, -0.014910226687788963, 0.05412788689136505, 0.021201221272349358, 0.03438936918973923, -0.024035369977355003, 0.004059402272105217, 0.03502044081687927, 0.022488625720143318, -0.02736932411789894, 0.03766028210520744, 0.017295513302087784, -0.0012525031343102455, -0.017278671264648438, -0.03313162922859192, 0.012665005400776863, 0.014525388367474079, 0.01660112477838993, 0.017458800226449966, 0.018277380615472794, 0.00319869793020

In [161]:
file_path = 'embedding.csv'  # Replace with your CSV file name
num_rows_to_print = 5  # Number of rows to print

with open(file_path, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)

    # Optional: Skip the header row if present
    # next(csv_reader)

    for i, row in enumerate(csv_reader):
        if i < num_rows_to_print:
            print(row)
        else:
            break

['1', "{'book': 1, 'page': 1, 'header': 'พระไตรปิฎกเล่มที่ ๐๑-๑ วินัยปิฎกที่ ๐๑ มหาวิภังค์ ภาค ๑', 'contents': ['พระวินัยปิฎก มหาวิภังค์ เวรัญชกัณฑ์ พระวินัยปิฎก มหาวิภังค์ ภาค ๑ _____________ ขอนอบน้อมพระผู้มีพระภาคอรหันตสัมมาสัมพุทธเจ้าพระองค์นั้น', 'เวรัญชกัณฑ์ เรื่องเวรัญชพราหมณ์', '[๑] สมัยนั้น พระผู้มีพระภาคพุทธเจ้าประทับอยู่ ณ ควงต้นสะเดาอันเป็น ที่อยู่ของนเฬรุยักษ์ เขตเมืองเวรัญชา พร้อมกับภิกษุสงฆ์หมู่ใหญ่ประมาณ ๕๐๐ รูป เวรัญชพราหมณ์ได้ฟังข่าวว่า ท่านพระสมณโคดม เป็นศากยบุตร เสด็จออกผนวช จากศากยตระกูล ประทับอยู่ ณ ควงต้นสะเดาอันเป็นที่อยู่ของนเฬรุยักษ์ เขตเมือง เวรัญชา พร้อมกับภิกษุสงฆ์หมู่ใหญ่ประมาณ ๕๐๐ รูป ท่านพระโคดมผู้เจริญนั้น มีกิตติศัพท์อันงามขจรไปอย่างนี้ว่า แม้เพราะเหตุนี้ พระผู้มีพระภาค พระองค์นั้น เป็นพระอรหันต์ ตรัสรู้ด้วยพระองค์เองโดยชอบ เพียบพร้อมด้วยวิชชาและจรณะ เสด็จไปดี รู้แจ้งโลก เป็นสารถีฝึกผู้ที่ควรฝึกได้อย่างยอดเยี่ยม เป็นศาสดาของ เทวดาและมนุษย์ทั้งหลาย เป็นพระพุทธเจ้า เป็นพระผู้มีพระภาค๑'], 'footnotes': ['เชิงอรรถ : ๑ พระพุทธคุณ ทั้ง ๙ บทนี้ แต่ละบทมีอรรถอเ